In [7]:
import re
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import pymongo
import json
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

In [2]:
client = pymongo.MongoClient('54.201.199.246', 27016)

wiki_db = client.wikipedia

wiki_col = wiki_db.my_collection

In [3]:
wiki_col.count()

1636

In [4]:
wiki_cursor = wiki_col.find()

In [5]:
next(wiki_cursor)['content']

{'*': '{{Multiple issues|{{refimprove|date=July 2017}}{{more footnotes|date=July 2017}}}}\n\n\'\'\'Data exploration\'\'\' is an approach similar to initial data analysis, whereby a data analyst uses visual exploration to understand what is in a dataset and the characteristics of the data, rather than through traditional data management systems<ref name="Foster">[https://www.fosteropenscience.eu/sites/default/files/pdf/2933.pdf FOSTER Open Science], Overview of Data Exploration Techniques: Stratos Idreos, Olga Papaemmonouil, Surajit Chaudhuri.</ref>. These characteristics can include size or amount of data, completeness of the data, correctness of the data, possible relationships amongst data elements or files/tables in the data.\n\nData exploration is typically conducted using a combination of automated and manual activities.<ref name="Foster" /><ref name="Stanford2011">[http://vis.stanford.edu/files/2011-Wrangler-CHI.pdf Stanford.edu], 2011 Wrangler: Interactive Visual Specification o